<a href="https://colab.research.google.com/github/matthog/MLHW/blob/main/HW7_MattHogan_801109363.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import time

# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# CIFAR-10 data preparation
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Baseline CNN
class BaselineCNN(nn.Module):
    def __init__(self):
        super(BaselineCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 8 * 8)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Extended CNN
class ExtendedCNN(nn.Module):
    def __init__(self):
        super(ExtendedCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(128 * 4 * 4, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        x = x.view(-1, 128 * 4 * 4)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Training and evaluation
def train_and_evaluate(model, epochs=200):
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    start_time = time.time()
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch {epoch + 1}/{epochs}, Loss: {running_loss / len(train_loader):.4f}")

    training_time = time.time() - start_time

    # Evaluation
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Training Time: {training_time:.2f}s, Test Accuracy: {accuracy:.2f}%")
    return training_time, running_loss / len(train_loader), accuracy

# Baseline CNN
print("Training Baseline CNN")
baseline_model = BaselineCNN()
baseline_results = train_and_evaluate(baseline_model)

# Extended CNN
print("\nTraining Extended CNN")
extended_model = ExtendedCNN()
extended_results = train_and_evaluate(extended_model)

# Report results
print("\nResults Comparison:")
print(f"Baseline CNN - Time: {baseline_results[0]:.2f}s, Loss: {baseline_results[1]:.4f}, Accuracy: {baseline_results[2]:.2f}%")
print(f"Extended CNN - Time: {extended_results[0]:.2f}s, Loss: {extended_results[1]:.4f}, Accuracy: {extended_results[2]:.2f}%")


100%|██████████| 170M/170M [00:03<00:00, 47.4MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Training Baseline CNN
Epoch 1/200, Loss: 1.3267
Epoch 2/200, Loss: 0.9497
Epoch 3/200, Loss: 0.7962
Epoch 4/200, Loss: 0.6877
Epoch 5/200, Loss: 0.5896
Epoch 6/200, Loss: 0.4952
Epoch 7/200, Loss: 0.4105
Epoch 8/200, Loss: 0.3297
Epoch 9/200, Loss: 0.2626
Epoch 10/200, Loss: 0.2045
Epoch 11/200, Loss: 0.1576
Epoch 12/200, Loss: 0.1255
Epoch 13/200, Loss: 0.1141
Epoch 14/200, Loss: 0.0931
Epoch 15/200, Loss: 0.0779
Epoch 16/200, Loss: 0.0767
Epoch 17/200, Loss: 0.0611
Epoch 18/200, Loss: 0.0608
Epoch 19/200, Loss: 0.0756
Epoch 20/200, Loss: 0.0554
Epoch 21/200, Loss: 0.0499
Epoch 22/200, Loss: 0.0597
Epoch 23/200, Loss: 0.0534
Epoch 24/200, Loss: 0.0532
Epoch 25/200, Loss: 0.0540
Epoch 26/200, Loss: 0.0407
Epoch 27/200, Loss: 0.0364
Epoch 28/200, Loss: 0.0527
Epoch 29/200, Loss: 0.0453
Epoch 30/200, Loss: 0.0406
Epoch 31/200, Loss: 0.0322
Epoch 32/200, Loss: 0.0476
Epoch 33/200, Loss: 0.0434
Epoch 3